<h2>Homework 5</h2>
M5086 &mdash; Stochastic Processes<BR>
Summer 2016<BR>
Tarleton State University<BR>
Prof. Scott Cook

On day 1, we looked at Persi Diaconis's paper called "The Markov Chain Monte Carlo Revolution."  He presented an MCMC technique to break 1-1 replacement codes.  Congratulations - we've now learned enough mathematics to actually do it oursevles.  So, let's do it.

Each of you have your own, unique ciphered message below.  They are all (randomly selected) extracts from a book that played a significant role in my childhood.

Your steps
1. Get a reference text (corpus). Load into Python.
- I gave you a complete list of the symbols that might appear in your message.  Appropriately clean any other symbols that appear in your corpus.
- Construct the frequency array
- Construct the plausibility function
- Pick an initial guess for the decipher
- Randomly walk through the space of all possible deciphers, moving toward more plausible one as prescribed by the Metropolis-Hastings algorithm
- Submit your deciphered quote

Here are some things to remember from the Diaconsis paper and stuff he doesn't talk much about.
- He uses letter pairs as the basic unit.  Call these 2-grams.  We might consider using longer basic units, $n$-grams.  In this description, I'll think about 2-grams.  But the ideas and code can be extended for $n$-grams (discussed in class).
- My personal prefernce is to work with numbers rather than letters.  Use text_to_numeric to convert from text to numbers.  This function replaces each letter by the index where that letter appears in "symbols."  Use numeric_to_text for the reverse.
- Covert the corpus from text to numbers using the text_to_numeric function.  
- Covert ciphered_text from text to numbers using the text_to_numeric function.
- A decipher is a bijection between the integers 0,1,...,num_symbols-1
- Use apply_decipher to apply a decipher
- plausibilty vs log_plausibility
  - Diaconis tells us to multiply the frequencies to get plausibilty.  But this is a product of a lot of small numbers.  We often bump into precision issues.
  - Recall the log of a product equals the sum of the logs.  So $$\mbox{plausibility}=\mbox{product of frequencies} = e^{\mbox{ln(product of frequencies)}}=e^{\mbox{sum(ln of frequencies)}}$$  Let $\mbox{log_plausibilty}=\mbox{sum(ln of frequencies)}$.  We work mainly with log_plausibility; this seems to nicely avoid the rounding issues.
  - If you need "real" plausibility, compute e^log_plausibility
- How to construct the reference matrix M
  - The (i,j) entry of M will refer to 2-letter sequence symbols[i] followed by symbols[j]
  - Construct M by:
    - Initialize a num_symbols by num_symbols array filled with **ONES**.  We use 1 because (as discussed earlier), we will log all the entries.  But ln(0) is undefined - creates errors.  Initializing with ONES avoids this issue and does not substantially impact the algorithm.
    - Loop through corpus_numeric, looking at 2-grams.  If you see $(i,j)$, add 1 to $M[i,j]$.
    - Normalize the rows so they sum to one. (For $n$-grams, normalize the last axis).
    - Let M = np.log(M) (reason discussed above)
- How to compute the log_plausibility
    - Loop through deciphered_numeric, looking at 2-grams.  If you see $(i,j)$, get $M[i,j]$.
    - **Add** these numbers.  That is log_plausibilty
    - If you need "real" plausibility, compute e^log_plausibility
- Now the MCMC part.  Here is how I did it.  I'm sure there are other ways.  
  - Create the next proposed decipher by chosing 2 entries of the current decipher at random and flipping them.
  - Let plaus_diff = log_plaus(proposed)-log_plaus(old).
      - If plaus_diff $\geq$ 0, accept move.
      - If plaus_diff < 0, then
          - Compute accept=e^plaus_diff
          - Draw r = unif(0,1)
          - If r <= accept, accept move.
          - If r > accept, reject move.  Unflip the 2 entries you flipped earlier.
- You may wany to run multiple simultaneous "samples".  For each, pick a random starting decipher and then evolve as described above.  Nest your loops so ALL samples move forward together (the "step" loop is outside, the "sample" loop is inside).  When any of the samples are "good enough", stop.  You only need one of them to find the right decipher.
  - Here is one way to establish "good enough."  Grab a random chunk of your corpus with the same length as your ciphered_text.  Compute its log_plausibilty (call it target_plaus).  This give you a sense of an average log_plaus for a correctly deciphered message.
  - After each step, let max_plaus be the largest log_plaus among your samples.  Terminate when e^(max_plaus-target_plaus) is close enough to 1.

In [1]:
from header import *
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import time

def support(v):
    return np.nonzero(v)[0]

def normalize_vec(v):
    tol = 0.001
    s = np.sum(v)
    if np.abs(s)<tol:
        s = v[support(v)][0] #if row sums to 0, divide by leftmost nonzero entry    
    return v/s

def normalize_rows(P):
    return np.apply_along_axis(normalize_vec,1,P)

def make_reference():
    f = open('hilarious_book.txt')
    corpus = f.read()
    f.close()
    corpus_len = len(corpus)

    """
    #Remove some symbols we probably don't need to worry about
    symbols = sorted(list(set(corpus.lower()))) #gets a list of the unique symbols in corupus
    print(symbols)
    corpus = corpus.replace('\n',' ')
    """
    symbols = sorted(list(set(corpus.lower()))) #gets a list of the unique symbols in corupus
    num_symbols = len(symbols)    
    return corpus, corpus_len, symbols, num_symbols

def text_to_numeric(text):
    return tuple(symbols.index(char.lower()) for char in text)

def numeric_to_text(numeric):
    s = (symbols[num] for num in numeric)
    return ''.join(s)

def apply_cipher(cipher,message_numeric):
    return  tuple(cipher[s] for s in message_numeric)

In [2]:
#Generates ciphered message - used by instructor
import numpy as np
message_len = 2000
corpus, corpus_len, symbols, num_symbols = make_reference()
print(symbols)
print(num_symbols)

start = np.random.randint(corpus_len-message_len)
plain_text = corpus[start:start+message_len]
plain_numeric = text_to_numeric(plain_text)
print(plain_text)

true_cipher = np.random.permutation(num_symbols)
ciphered_numeric = apply_cipher(true_cipher,plain_numeric)
ciphered_text = numeric_to_text(ciphered_numeric) 
print("\n\n" + ciphered_text)

true_decipher = np.argsort(true_cipher)
deciphered_numeric = apply_cipher(true_decipher,ciphered_numeric)
deciphered_text = numeric_to_text(deciphered_numeric) 
print("\n\n" + deciphered_text)

[' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
49
onymity and nothing would transpire. The planet's surface  was  blurred  by  time, by the slow movement of the thin stagnant air that had crept across it for century upon century.  Clearly, it was very very old.  A moment of doubt came to Ford as he watched the  grey  landscape move  beneath  them.  The immensity of time worried him, he could feel it as a presence. He cleared his throat.  "Well, even supposing it is ..."  "It is," said Zaphod.  "Which it isn't," continued Ford.  "What  do  you  want  with  it anyway? There's nothing there." "Not on the surface," said Zaphod.  "Alright, just supposing there's something. I take it you're  not here for the sheer industrial archaeology of it all. What are you after?"  One of Zaphod's heads looke

In [3]:
#Generates ciphered message - used by instructor
import numpy as np
import json
message_len = 2000
corpus, corpus_len, symbols, num_symbols = make_reference()
print("All legal symbols")
print(symbols)
print(num_symbols)

Students = dict()
Names = ['booth','brown','culver','hutrya','lindsey','mccoy','saunders','stewart','tintera','tomlinson','upreti']
for name in Names:
    start = np.random.randint(corpus_len-message_len)
    plain_text = corpus[start:start+message_len]
    plain_numeric = text_to_numeric(plain_text)

    true_cipher = np.random.permutation(num_symbols)
    ciphered_numeric = apply_cipher(true_cipher,plain_numeric)
    ciphered_text = numeric_to_text(ciphered_numeric) 
    print("\n\n\n"+name)    
    print(ciphered_text)
    Students[name] = {'start':start,'cipher':str(true_cipher.tolist()),'plain_text':plain_text}

f = open('message.txt','w')
json.dump(Students,f,sort_keys=True,indent=4)
f.close()

f = open('message.txt','r')
read = json.load(f)

All legal symbols
[' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
49



booth
:77t8q7';:7f8q/06'87t8;9 89 /:;87t8i7f38f77z 38f6z 88/88) :t q;fr88q7'" ';67'/f88,)/q ,96)88 uq );88;9/;886;82/,8) :t q;fr8qf /'80 q/5, 86;82/,8,78' 2w8,7p 87t8;9 8q7';:7f88, /;,89/3'(;889/388;9 88)f/,;6q882:/))6'i88;/z '87tt8r ;w8;9 8q/06'82/,8p7,;fr88296; h8870f7'ih88/'388/075;88;9 88,61 887t888/888,p/ff6,98: ,;/5:/';w886'88t/q;886;882/,'(;88) :t q;fr870f7'ik8;9 8;278f7'i82/ff,82 : 8:/z 38:75'386'8/8,f6i9;8)/:/ff f8q5:" h88/'388/ff88;9 8/'if ,88/'388q7:' :,882 : 8q7';75: 386'8 uq6;6'ifr8q95'zr8,9/) ,w8;9 8;:5;987t8;9 8p/;; :86,8;9/;86;8275f389/" 80  '8/88i: /;883 /f8,6p)f :88/'388p7: 88):/q;6q/f88;788056f38;9 8q/06'8/,8/'87:36'/:r8;9:  ?36p ',67'/f870f7'i8:7ph805;8;9 '8;9 83 ,6i' :,88275f3889/" 8i7;88p6, :/0f w8/